<a href="https://colab.research.google.com/github/cyFou/testColab/blob/main/testTRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#source de ce test :
# https://moazharu.medium.com/building-tiny-recursive-model-from-scratch-when-tiny-networks-beat-giants-at-their-own-game-68d9df9e1fdb
# https://github.com/SamsungSAILMontreal/TinyRecursiveModels/blob/main/dataset/build_arc_dataset.py

In [1]:
!git clone https://github.com/SamsungSAILMontreal/TinyRecursiveModels.git

Cloning into 'TinyRecursiveModels'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 75 (delta 3), reused 2 (delta 2), pack-reused 69 (from 2)
Receiving objects: 100% (75/75), 1.25 MiB | 6.78 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
!pip install argdantic
!pip install coolname
!pip install hydra-core


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.1 MB/s eta 0:00:00


In [3]:
%cd /content/TinyRecursiveModels

# Téléchargement du dataset arc-agi-1 en utilisant le script du dépôt TinyRecursiveModels
!python -m dataset.build_arc_dataset \
  --input-file-prefix kaggle/combined/arc-agi \
  --output-dir data/arc1concept-aug-1000 \
  --subsets training evaluation concept \
  --test-set-name evaluatio

/content/TinyRecursiveModels
[Puzzle 8be77c9e] augmentation not full, only 72
[Puzzle 4258a5f9] augmentation not full, only 576
[Puzzle 3618c87e] augmentation not full, only 575
[Puzzle 2281f1f4] augmentation not full, only 576
[Puzzle 3906de3d] augmentation not full, only 576
[Puzzle aedd82e4] augmentation not full, only 576
[Puzzle 4612dd53] augmentation not full, only 575
[Puzzle 28e73c20] augmentation not full, only 72
[Puzzle f15e1fac] augmentation not full, only 576
[Puzzle 44d8ac46] augmentation not full, only 576
[Puzzle dc433765] augmentation not full, only 575
[Puzzle a5313dff] augmentation not full, only 576
[Puzzle 3f7978a0] augmentation not full, only 576
[Puzzle d4f3cd78] augmentation not full, only 576
[Puzzle 760b3cac] augmentation not full, only 576
[Puzzle cce03e0d] augmentation not full, only 576
[Puzzle ef135b50] augmentation not full, only 576
[Puzzle a3df8b1e] augmentation not full, only 72
[Puzzle 6855a6e4] augmentation not full, only 576
[Puzzle 05f2a901] augmen

In [4]:
%cd /content/TinyRecursiveModels

!python dataset/build_sudoku_dataset.py --output-dir data/sudoku-extreme-1k-aug-1000  --subsample-size 10 --num-aug 1000  # 1000 examples, 1000 augments

train.csv: 100% 719M/719M [00:05<00:00, 131MB/s]
100% 10/10 [00:02<00:00,  3.70it/s]
test.csv: 100% 79.4M/79.4M [00:01<00:00, 40.6MB/s]
100% 422786/422786 [00:00<00:00, 1673317.32it/s]


In [ ]:
!run_name="pretrain_mlp_t_sudoku"
!python pretrain.py \
arch=trm \
data_paths="[data/sudoku-extreme-1k-aug-1000]" \
evaluators="[]" \
epochs=50000 eval_interval=5000 \
lr=1e-4 puzzle_emb_lr=1e-4 weight_decay=1.0 puzzle_emb_weight_decay=1.0 \
arch.mlp_t=True arch.pos_encodings=none \
arch.L_layers=2 \
arch.H_cycles=3 arch.L_cycles=6 \
+run_name=${run_name} ema=True

Traceback (most recent call last):
  File "/content/TinyRecursiveModels/pretrain.py", line 20, in <module>
    from adam_atan2 import AdamATan2
ModuleNotFoundError: No module named 'adam_atan2'


In [5]:
from torch.utils.data import DataLoader
from puzzle_dataset import PuzzleDataset, PuzzleDatasetConfig, PuzzleDatasetMetadata



In [16]:
def create_dataloader(data_paths, split: str, rank: int, world_size: int, **kwargs):
    dataset = PuzzleDataset(PuzzleDatasetConfig(
        seed=0,
        dataset_paths=data_paths,
        rank=rank,
        num_replicas=world_size,
        **kwargs
    ), split=split)
    dataloader = DataLoader(
        dataset,
        batch_size=None,
        num_workers=1,
        prefetch_factor=8,
        pin_memory=True,
        persistent_workers=True
    )
    return dataloader, dataset.metadata

In [17]:
RANK = 0
WORLD_SIZE = 1
CPU_PROCESS_GROUP = None
eval_interval = 10000
global_batch_size = 768
data_paths = ['data/sudoku-extreme-1k-aug-1000']

train_loader, train_metadata = create_dataloader(data_paths,"train", test_set_mode=False, epochs_per_iter=eval_interval, global_batch_size=global_batch_size, rank=RANK, world_size=WORLD_SIZE)

In [22]:
N = 1
dataiter = iter(train_loader)
set_name, batch, global_batch_size = next(dataiter)
print(len(batch),global_batch_size)
print(batch)

# for set_name, batch, global_batch_size in train_loader:
  # print(batch)


3 768
{'inputs': tensor([[ 1,  7,  9,  ...,  1,  1,  1],
        [10,  1,  1,  ...,  1,  1,  2],
        [ 1,  8,  1,  ...,  8,  1,  1],
        ...,
        [ 1,  1,  3,  ...,  1,  1,  1],
        [ 1,  4,  1,  ...,  1,  8,  1],
        [ 1,  1,  1,  ...,  1,  6,  1]], dtype=torch.int32), 'labels': tensor([[ 6,  7,  9,  ...,  5, 10,  9],
        [10,  2,  3,  ...,  7,  6,  2],
        [ 9,  8,  5,  ...,  8,  5,  7],
        ...,
        [ 4,  8,  3,  ...,  3,  4,  6],
        [10,  4,  9,  ...,  5,  8, 10],
        [10,  2,  6,  ...,  2,  6,  8]], dtype=torch.int32), 'puzzle_identifiers': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
   